## TODO
* check for last short text part

In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoModel
from datasets import load_dataset
from tqdm import tqdm
import random
from huggingface_hub import login
import pandas as pd
import numpy as np
import json
import gc
import os
import re

---

# Openai embeddings

In [2]:
import openai

In [3]:
with open(r"D:\Programming\My_reps\OpenAI_API_my.txt", 'r') as f:
    openai_key = f.readlines()[0]
client = openai.OpenAI(api_key=openai_key)

def get_embedding(text, model='text-embedding-3-small'):
    if isinstance(text, str):
        text = [text]
    out = client.embeddings.create(input = text, model=model)
    return [o.embedding for o in out.data]

In [9]:
vector = get_embedding('cypher language')
np.array(vector).shape

(1, 1536)

In [2]:
# generate embeddings
folder = r"D:\Programming\My_reps\Similarity_search_audio\tmp_work_files\texts_from_audio"
output_folder = r"D:\Programming\My_reps\Similarity_search_audio\tmp_work_files\embeddings_for_texts_openai"

In [3]:
with open(os.path.join(folder, 'processed_files.txt'), 'r') as f:
    processed_files = [p.replace('\n', '') for p in f.readlines()]
    
def update_processed(filename):
    global processed_files
    processed_files.append(filename)
    with open(os.path.join(folder, 'processed_files.txt'), 'a') as f:
        f.writelines(processed_files[-1] + '\n')

len(processed_files)

50

In [10]:
files = tqdm([o for o in os.listdir(folder) if o.endswith('csv')])
for filename in files:
    files.set_description(filename[:40])
    youtube_link = re.search(r"(\[[^\]]+\]).csv$", filename).group(1)
    if youtube_link in processed_files:
        continue
        
    file_path = os.path.join(folder, filename)
    ds = pd.read_csv(file_path)
    texts = ds.loc[:, 'text'].tolist()
    # summs = ds.loc[:, 'summarization'].tolist()

    embeddings_texts = get_embedding(texts)

    np.save(os.path.join(output_folder, os.path.splitext(filename)[0] + '.text.npy'), embeddings_texts)
    # np.save(os.path.join(output_folder, os.path.splitext(filename)[0] + '.summarization.npy'), embeddings_texts)

    update_processed(youtube_link)

Рав Дов Бегон. Недельная глава Хукат. #1: 100%|████████████████████████████████████████| 50/50 [00:33<00:00,  1.49it/s]


In [11]:
!start .

In [8]:
filename

'Основные ценности евреев. Талмуд по-русски. Рав Йосеф Менделевич #11-09-2024# [kSlA3w1wrxc].csv'

---

# Test

In [2]:
with open('D:\Programming\My_reps\Hugging_face_token_my.txt', 'r') as f:
    hf_token = f.readlines()[0]
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Otniel\.cache\huggingface\token
Login successful


In [3]:
def average_pool(last_hidden_states: torch.Tensor,
                 attention_mask: torch.Tensor) -> torch.Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

In [4]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large-instruct')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large-instruct', torch_dtype=torch.bfloat16, trust_remote_code=True, 
                                  device_map="cuda").eval()

In [12]:
vector_base = []

dss = []
folder = r"D:\Programming\My_experiments\Audio\texts_from_audio"
embd_folder = r"D:\Programming\My_experiments\Audio\embeddings_for_texts_openai"

for filename in tqdm([o for o in os.listdir(folder) if o.endswith('csv')]):
    file_path = os.path.join(folder, filename)
    dss.append(pd.read_csv(file_path))
    dss[-1]['filename'] = filename
    embd_file_name = os.path.join(embd_folder, os.path.splitext(filename)[0] + ".text.npy")
    vector_base.append(torch.tensor(np.load(embd_file_name)))

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 160.66it/s]


In [13]:
dss[0]

,start,end,text,filename
0,0.00,115.36,"Доброе утро. Я хотел повторить сперва, прежде ...",В жизни логика не работает. Урок №4. Талмуд по...
1,115.36,209.88,но то что есть мы понимаем мы царь на все я на...,В жизни логика не работает. Урок №4. Талмуд по...
2,209.88,335.90,"Но надо проверить, с какого языка это взято, в...",В жизни логика не работает. Урок №4. Талмуд по...
3,335.90,478.84,входило в контекст теперь мы читаем повторение...,В жизни логика не работает. Урок №4. Талмуд по...
4,478.84,590.62,случае это не то и не сел а это как раз пробел...,В жизни логика не работает. Урок №4. Талмуд по...
5,590.62,713.38,и вы ему на м этого ему на да и потом будет ещ...,В жизни логика не работает. Урок №4. Талмуд по...
6,713.38,834.60,"И оно, естественно, будет позволено, вставлени...",В жизни логика не работает. Урок №4. Талмуд по...
7,834.60,920.10,бы аллаха гуманной говорит это как бы не когда...,В жизни логика не работает. Урок №4. Талмуд по...
8,920.10,1077.00,"Вот давайте сравним эти две, как бы сравним мн...",В жизни логика не работает. Урок №4. Талмуд по...
9,1077.00,1190.68,"И дальше мы разбираем следующее. Теперь, «бепр...",В жизни логика не работает. Урок №4. Талмуд по...


In [14]:
vector_base[0].shape

torch.Size([36, 1536])

In [15]:
# query = ['как учить талмуд']
query = ['капля молока попала в мясной суп, что делать?']

In [16]:
# Openai|
query_embeddings = get_embedding(query)

In [17]:
results = []
for i, vect in enumerate(vector_base):
    scores = (torch.tensor(query_embeddings) @ vect.to(torch.float32).T) * 100
    results.extend([[score, i, j] for j, score in enumerate(scores.numpy()[0])])

In [18]:
results

[[10.295246, 0, 0],
 [3.9045055, 0, 1],
 [10.140442, 0, 2],
 [17.244217, 0, 3],
 [13.588962, 0, 4],
 [20.201286, 0, 5],
 [19.85116, 0, 6],
 [9.840572, 0, 7],
 [16.500408, 0, 8],
 [18.69809, 0, 9],
 [16.274666, 0, 10],
 [21.0566, 0, 11],
 [12.6378565, 0, 12],
 [9.304233, 0, 13],
 [14.980125, 0, 14],
 [13.957614, 0, 15],
 [16.691925, 0, 16],
 [14.185175, 0, 17],
 [18.330616, 0, 18],
 [6.304957, 0, 19],
 [13.922745, 0, 20],
 [21.108644, 0, 21],
 [15.918189, 0, 22],
 [22.415407, 0, 23],
 [13.307917, 0, 24],
 [12.937376, 0, 25],
 [13.684557, 0, 26],
 [16.761559, 0, 27],
 [11.883084, 0, 28],
 [15.466306, 0, 29],
 [10.819404, 0, 30],
 [5.2762117, 0, 31],
 [17.075941, 0, 32],
 [10.809907, 0, 33],
 [13.7271185, 0, 34],
 [14.610626, 0, 35],
 [10.445389, 1, 0],
 [15.309557, 1, 1],
 [10.267342, 1, 2],
 [13.275707, 1, 3],
 [11.153038, 1, 4],
 [15.907556, 1, 5],
 [14.20432, 1, 6],
 [15.763485, 1, 7],
 [13.235879, 1, 8],
 [11.520236, 1, 9],
 [22.665325, 1, 10],
 [14.304921, 1, 11],
 [10.791773, 1, 12

In [19]:
sorted(results, key=lambda x:-x[0])

[[36.019146, 42, 10],
 [35.382084, 42, 19],
 [32.754852, 42, 23],
 [32.5247, 2, 3],
 [31.996082, 42, 18],
 [30.929598, 42, 14],
 [30.715126, 34, 2],
 [30.353863, 36, 19],
 [30.146343, 42, 8],
 [29.952257, 35, 19],
 [29.547506, 35, 17],
 [28.928116, 2, 16],
 [28.76509, 33, 4],
 [28.412163, 42, 20],
 [28.271288, 36, 21],
 [27.777538, 25, 12],
 [27.562958, 25, 18],
 [27.363861, 34, 15],
 [27.297037, 34, 7],
 [27.293152, 45, 7],
 [27.162897, 25, 16],
 [26.940859, 40, 24],
 [26.763496, 42, 22],
 [26.645655, 1, 23],
 [26.533758, 47, 29],
 [26.193792, 39, 17],
 [26.178062, 38, 17],
 [25.96682, 36, 12],
 [25.866562, 48, 14],
 [25.865273, 17, 12],
 [25.738897, 47, 5],
 [25.587452, 5, 28],
 [25.468212, 34, 19],
 [25.376091, 33, 20],
 [25.272427, 36, 11],
 [25.20099, 6, 5],
 [25.074148, 47, 7],
 [24.989838, 47, 16],
 [24.973724, 18, 24],
 [24.807203, 34, 20],
 [24.796654, 36, 4],
 [24.731611, 40, 16],
 [24.729738, 25, 15],
 [24.720188, 33, 1],
 [24.712122, 35, 18],
 [24.646338, 24, 13],
 [24.6382

In [20]:
for res in sorted(results, key=lambda x:-x[0])[:5]:
    print(res, dss[res[1]].loc[res[2], 'filename'], '\n', dss[res[1]].loc[res[2], 'text'], '\n')

[36.019146, 42, 10] Рав Элиягу Бережинский. Чуть - чуть про кашрут (7). Еврейское вино. [VRZzTTDbls4].csv 
 абсолютно значит единственное что здесь нужно чтобы пропорция была довольно большая то есть изюмом и добавляем не горстку на кастрюлю а нужно на кастрюлю 6 литровую кастрюлю килограмм с хвостиком или килограмм изюмой чуть-чуть не долить кастрюлю сварил компот у тебя есть сейчас там 5 литров или 4,5 литра вина для Песаха. Тебе не хватит? Хватит. Можно пить даже компоты, он тоже будет буре при Агафе. Такой компот, если у тебя есть время, и ты это делаешь заранее, сделать это за месяц, потому что пока оно забродит, это где-то недели 3-4 берет. На месяц ты можешь сделать вино из изюма, а можешь сделать ускоренный вариант. Взял изюм, сварил его, дал ему остыть и выпил вечером остывший компот, который ты в этот день сделал. Можно подождать, пока компот забродит. Такой вариант, в принципе, тоже может быть, но это сложный вариант, потому что после полноценной варки, как мы уже сказали,

# Example Gemma2

In [14]:
# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'how much protein should a female eat'),
    get_detailed_instruct(task, 'big objects in space like quasars'),
    get_detailed_instruct(task, 'большие объекты в космосе такие как квазары'),
    get_detailed_instruct(task, '南瓜的家常做法')
]
# No need to add instruction for retrieval documents
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"
]
input_texts = queries + documents

In [15]:
# Tokenize the input texts
with torch.no_grad():
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**batch_dict.to('cuda'))
    embeddings = average_pool(outputs.last_hidden_state.cpu(), batch_dict['attention_mask'].cpu())
    torch.cuda.empty_cache()
    gc.collect()

In [16]:
# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:4] @ embeddings[4:].T) * 100
print(scores.tolist())

[[92.0, 67.5], [70.5, 63.0], [68.0, 64.5], [70.5, 92.5]]


# Example USER-bge-m3

In [5]:
tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3")
model = AutoModel.from_pretrained("deepvk/USER-bge-m3", torch_dtype=torch.bfloat16, trust_remote_code=True, 
                                  device_map="cuda").eval()

In [6]:
input_texts = [
  "Когда был спущен на воду первый миноносец «Спокойный»?",
  "Есть ли нефть в Удмуртии?",
  'большие объекты в космосе такие как квазары',
  "Спокойный (эсминец)\nЗачислен в списки ВМФ СССР 19 августа 1952 года.",
  "Нефтепоисковые работы в Удмуртии были начаты сразу после Второй мировой войны в 1945 году и продолжаются по сей день. Добыча нефти началась в 1967 году."
]

In [7]:
with torch.no_grad():
    encoded_input = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**encoded_input.to('cuda'))
    sentence_embeddings = outputs[0][:, 0].to('cpu')
    torch.cuda.empty_cache()
    gc.collect()

In [8]:
# normalize embeddings
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
scores = (sentence_embeddings[:3] @ sentence_embeddings[3:].T)
scores

tensor([[0.5547, 0.3027],
        [0.1689, 0.7109],
        [0.1406, 0.1885]], dtype=torch.bfloat16)

# Generate embeddings

In [24]:
# generate embeddings
folder = r"D:\Programming\My_experiments\Audio\texts_from_audio"
output_folder = r"D:\Programming\My_experiments\Audio\embeddings_for_texts"

for filename in tqdm(os.listdir(folder)):
    file_path = os.path.join(folder, filename)
    ds = pd.read_csv(file_path)
    texts = ds.loc[:, 'text'].tolist()
    summs = ds.loc[:, 'summarization'].tolist()

    with torch.no_grad():
        encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**encoded_input.to('cuda'))
        embeddings_texts = outputs[0][:, 0].to('cpu')
        embeddings_texts = torch.nn.functional.normalize(embeddings_texts, p=2, dim=1).to(torch.float16).numpy()

        encoded_input = tokenizer(summs, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**encoded_input.to('cuda'))
        embeddings_summs = outputs[0][:, 0].to('cpu')
        embeddings_summs = torch.nn.functional.normalize(embeddings_summs, p=2, dim=1).to(torch.float16).numpy()
        
        torch.cuda.empty_cache()
        gc.collect()

    np.save(os.path.join(output_folder, os.path.splitext(filename)[0] + '.text.npy'), embeddings_texts)
    np.save(os.path.join(output_folder, os.path.splitext(filename)[0] + '.summarization.npy'), embeddings_texts)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:20<00:00,  2.46it/s]
